In [1]:
%reload_kedro

2023-10-10 01:09:46,800 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: C:\Users\Dushku\PycharmProjects\monash-datahack-2023
2023-10-10 01:09:46,929 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-10-10 01:09:46,950 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-telemetry-0.2.5
2023-10-10 01:09:47,004 - kedro.extras.extensions.ipython - INFO - ** Kedro project Monash datahack 2023
2023-10-10 01:09:47,005 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


C:\Users\Dushku\PycharmProjects\monash-datahack-2023\venv\lib\site-packages\kedro\framework\context\context.py:344: UserWarning: Credentials not found in your Kedro project config.
No files found in ['C:\\Users\\Dushku\\PycharmProjects\\monash-datahack-2023\\conf\\base', 'C:\\Users\\Dushku\\PycharmProjects\\monash-datahack-2023\\conf\\local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [2]:
df = catalog.load('preprocessed_data_not_recycled')

2023-10-10 01:09:47,025 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_not_recycled` (CSVDataSet)...


In [3]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Management', 'Fate', 'y', 'ds',
       'Sub_stream_name', 'Sub_stream_index'],
      dtype='object')

In [4]:
df = df[['Jurisdiction', 
        'Category', 
        'Type', 
        'Management', 
        'Fate', 
        'y', 
        'Sub_stream_name',
        'ds']]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33693 entries, 0 to 33692
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Jurisdiction     33693 non-null  object
 1   Category         33693 non-null  object
 2   Type             33693 non-null  object
 3   Management       33693 non-null  object
 4   Fate             33693 non-null  object
 5   y                33693 non-null  int64 
 6   Sub_stream_name  33693 non-null  object
 7   ds               33693 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


In [6]:
df['y'] = df['y'].astype(int)

In [7]:
# List of columns to exclude
exclude_columns = ['y', 'ds_encoded', ]

# Create a dictionary to store unique values for each column
unique_values_dict = {}

# Loop through columns and get unique values
for column in df.columns:
    if column not in exclude_columns:
        unique_values_dict[column] = df[column].unique()

# Print unique values for each column
for column, unique_values in unique_values_dict.items():
    print(f"Unique values for {column}:")
    print(unique_values)
    print()

Unique values for Jurisdiction:
['ACT' 'NSW' 'NT' 'Qld' 'SA' 'Tas' 'Vic' 'WA']

Unique values for Category:
['Biosolids' 'Building and demolition materials' 'Hazardous wastes'
 'Organics' 'Ash' 'Metals' 'Plastics' 'Unclassified materials'
 'Textiles, leather & rubber (excl. tyres)']

Unique values for Type:
['Biosolids'
 'Soil, sand and rock not contaminated above any threshold requiring classification as contaminated soils (N120)'
 'Acids (B)' 'Alkalis (C)' 'Asbestos (N220)'
 'Clinical and pharmaceutical (R)' 'Contaminated soils (N120)'
 'Food-derived hazardous wastes (K100, K110)' 'Inorganic chemicals (D)'
 'Oils (J)' 'Organic chemicals (M)' 'Organic solvents (G)' 'Other'
 'Other hazardous organic wastes (K140, K190)'
 'Other miscellaneous (other T)' 'Other soil/sludges (other N)'
 'Paints, resins, inks, organic sludges (F)' 'Pesticides (H)'
 'Plating and heat treatment (A)' 'Reactive chemicals (E)' 'Tyres (T140)'
 'Food organics' 'Garden organics' 'Other organics' 'Timber' 'Ash'
 'I

In [8]:
import datetime as dt
import pandas as pd

# Convert the 'date_column' to datetime and then extract the year
df['Year'] = pd.to_datetime(df['ds']).dt.year
df = df.drop('ds', axis=1)

In [9]:
from sklearn.preprocessing import LabelEncoder

# Encoding the categorical values
# Initialize the LabelEncoder for each categorical column
label_encoders = {}

# Columns to encode
encode_columns = ['Jurisdiction',
        'Sub_stream_name',
        'Category', 
        'Type', 
        'Management', 
        'Fate']

for col in encode_columns:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

In [10]:
import joblib
import os

# Define the folder name within your project
folder_name = '/encoders'

# Create the full path to the folder
folder_path = os.path.join(os.getcwd(), folder_name)

# Ensure that the folder exists; if not, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save the LabelEncoder objects to separate files
for col, encoder in label_encoders.items():
    encoder_filename = f'{col}_label_encoder.pkl'
    encoder_path = os.path.join(folder_path, encoder_filename)
    print(encoder_path)
    joblib.dump(encoder, encoder_path)

C:/encoders\Jurisdiction_label_encoder.pkl
C:/encoders\Sub_stream_name_label_encoder.pkl
C:/encoders\Category_label_encoder.pkl
C:/encoders\Type_label_encoder.pkl
C:/encoders\Management_label_encoder.pkl
C:/encoders\Fate_label_encoder.pkl


In [11]:
df.head()

,Jurisdiction,Category,Type,Management,Fate,y,Sub_stream_name,Year
0,0,1,6,2,0,20186,2,2021
1,0,1,6,2,0,20186,4,2021
2,0,2,37,4,3,133987,0,2021
3,0,2,37,4,3,133987,4,2021
4,0,3,0,0,1,0,0,2021


In [12]:
catalog.save("encoded_data_not_recycled", df)

2023-10-10 01:09:47,495 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_not_recycled` (CSVDataSet)...
